In [21]:
%load_ext autoreload
%autoreload 2

%pip install -r ../../requirements.txt

Note: you may need to restart the kernel to use updated packages.


El sistema no puede encontrar la ruta especificada.


In [22]:
import sys

# Se agrega el path para poder importar los modulos locales
if "../" not in sys.path:
    sys.path.append("../")

import pandas as pd
import numpy as np
from utils import one_hot_encode


In [23]:
flights = pd.read_parquet("../../resources/flightstest_clean.parquet")
flights2 = pd.read_parquet("../../resources/flights_clean.parquet")
flights.shape


(10193, 16)

In [24]:
flights2.shape

(120224, 16)

In [32]:
flights.columns==flights2.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

La variable real a predecir el modelo es la cantida de pasajeros de un vuelo individual

---

In [25]:
X = flights.drop(columns=["Passengers", "Flight_ID", "Bookings"])
X.head()

,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Mes,Semana,DiaSemana,Hora,HourDuration
0,XA-VIY,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2024-01-01 00:05:00,2024-01-01 05:05:00,186,1,1,1,0,5.000000
1,XA-VBT,BM,AO,Ciudad Principal,Ciudad Fronteriza,2024-01-01 00:30:00,2024-01-01 05:25:00,220,1,1,1,0,4.916667
2,XA-VAA,BM,AD,MX Amigos y Familia,Ciudad Fronteriza,2024-01-01 01:00:00,2024-01-01 05:50:00,186,1,1,1,1,4.833333
3,XA-VBN,AT,BM,Ciudad Fronteriza,Ciudad Principal,2024-01-01 05:00:00,2024-01-01 06:45:00,220,1,1,1,5,1.750000
4,XA-VYF,AT,BG,MX Amigos y Familia,Ciudad Principal,2024-01-01 05:30:00,2024-01-01 07:10:00,180,1,1,1,5,1.666667


Transformación de los datos categoricos a utilizar a variables numéricos

In [26]:
for col, prefix in zip(
    [
        "Aeronave",
        "DepartureStation",
        "ArrivalStation",
        "Destination_Type",
        "Origin_Type",
    ],
    [
        "aeronave",
        "departure",
        "arrival",
        "destino",
        "origen",
    ],
):
    X = one_hot_encode(X, column=col, prefix=prefix)

X.head()

,STD,STA,Capacity,Mes,Semana,DiaSemana,Hora,HourDuration,aeronave_9h-amm,aeronave_9h-amp,...,destino_ciudad_fronteriza,destino_ciudad_principal,destino_ecoturismo,destino_mx_amigos_y_familia,destino_playa,origen_ciudad_fronteriza,origen_ciudad_principal,origen_ecoturismo,origen_mx_amigos_y_familia,origen_playa
0,2024-01-01 00:05:00,2024-01-01 05:05:00,186,1,1,1,0,5.000000,0,0,...,0,0,0,1,0,1,0,0,0,0
1,2024-01-01 00:30:00,2024-01-01 05:25:00,220,1,1,1,0,4.916667,0,0,...,0,1,0,0,0,1,0,0,0,0
2,2024-01-01 01:00:00,2024-01-01 05:50:00,186,1,1,1,1,4.833333,0,0,...,0,0,0,1,0,1,0,0,0,0
3,2024-01-01 05:00:00,2024-01-01 06:45:00,220,1,1,1,5,1.750000,0,0,...,1,0,0,0,0,0,1,0,0,0
4,2024-01-01 05:30:00,2024-01-01 07:10:00,180,1,1,1,5,1.666667,0,0,...,0,0,0,1,0,0,1,0,0,0


In [27]:
X["Dia"] = X["STD"].dt.day_of_year
X = X.drop(columns=["STD"])
X.head()

,STA,Capacity,Mes,Semana,DiaSemana,Hora,HourDuration,aeronave_9h-amm,aeronave_9h-amp,aeronave_9h-amv,...,destino_ciudad_principal,destino_ecoturismo,destino_mx_amigos_y_familia,destino_playa,origen_ciudad_fronteriza,origen_ciudad_principal,origen_ecoturismo,origen_mx_amigos_y_familia,origen_playa,Dia
0,2024-01-01 05:05:00,186,1,1,1,0,5.000000,0,0,0,...,0,0,1,0,1,0,0,0,0,1
1,2024-01-01 05:25:00,220,1,1,1,0,4.916667,0,0,0,...,1,0,0,0,1,0,0,0,0,1
2,2024-01-01 05:50:00,186,1,1,1,1,4.833333,0,0,0,...,0,0,1,0,1,0,0,0,0,1
3,2024-01-01 06:45:00,220,1,1,1,5,1.750000,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,2024-01-01 07:10:00,180,1,1,1,5,1.666667,0,0,0,...,0,0,1,0,0,1,0,0,0,1


---

Se obtienen esta fecha del [calendario escolar de la SEP](https://calendarioescolar.sep.gob.mx)

In [28]:
dias_feriado = [
    1,  # Año Nuevo
    36,
    78,
    122,
    136,
    260,
    323,
    360,
    365,  # Año Nuevo
]

X["DiasAFeriadoCercano"] = X["Dia"].apply(
    lambda x: min([abs(x - feriado) for feriado in dias_feriado])
)

X.head()

,STA,Capacity,Mes,Semana,DiaSemana,Hora,HourDuration,aeronave_9h-amm,aeronave_9h-amp,aeronave_9h-amv,...,destino_ecoturismo,destino_mx_amigos_y_familia,destino_playa,origen_ciudad_fronteriza,origen_ciudad_principal,origen_ecoturismo,origen_mx_amigos_y_familia,origen_playa,Dia,DiasAFeriadoCercano
0,2024-01-01 05:05:00,186,1,1,1,0,5.000000,0,0,0,...,0,1,0,1,0,0,0,0,1,0
1,2024-01-01 05:25:00,220,1,1,1,0,4.916667,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,2024-01-01 05:50:00,186,1,1,1,1,4.833333,0,0,0,...,0,1,0,1,0,0,0,0,1,0
3,2024-01-01 06:45:00,220,1,1,1,5,1.750000,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,2024-01-01 07:10:00,180,1,1,1,5,1.666667,0,0,0,...,0,1,0,0,1,0,0,0,1,0


In [29]:
# Dias entre 15 de julio y 28 de agosto
vacaciones_verano = (X["Dia"] >= 197) & (X["Dia"] <= 241)
# Dias entre 18 de diciembre y 5 de enero
vacaciones_invierno = (X["Dia"] >= 353) & (X["Dia"] <= 5)
# Dias entre 25 de marzo y 5 de abril
vacaciones_diatrabajo = (X["Dia"] >= 85) & (X["Dia"] <= 96)

X["VacacionesEscolares"] = np.where(
    (vacaciones_verano) | (vacaciones_invierno) | (vacaciones_diatrabajo), 1, 0
)
X.head()

,STA,Capacity,Mes,Semana,DiaSemana,Hora,HourDuration,aeronave_9h-amm,aeronave_9h-amp,aeronave_9h-amv,...,destino_mx_amigos_y_familia,destino_playa,origen_ciudad_fronteriza,origen_ciudad_principal,origen_ecoturismo,origen_mx_amigos_y_familia,origen_playa,Dia,DiasAFeriadoCercano,VacacionesEscolares
0,2024-01-01 05:05:00,186,1,1,1,0,5.000000,0,0,0,...,1,0,1,0,0,0,0,1,0,0
1,2024-01-01 05:25:00,220,1,1,1,0,4.916667,0,0,0,...,0,0,1,0,0,0,0,1,0,0
2,2024-01-01 05:50:00,186,1,1,1,1,4.833333,0,0,0,...,1,0,1,0,0,0,0,1,0,0
3,2024-01-01 06:45:00,220,1,1,1,5,1.750000,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,2024-01-01 07:10:00,180,1,1,1,5,1.666667,0,0,0,...,1,0,0,1,0,0,0,1,0,0


In [30]:
X = X.drop(columns=["STA"])

In [31]:
X.to_parquet(f"data/variablesTest.parquet", index=False)

In [34]:
X.columns

Index(['Capacity', 'Mes', 'Semana', 'DiaSemana', 'Hora', 'HourDuration',
       'aeronave_9h-amm', 'aeronave_9h-amp', 'aeronave_9h-amv',
       'aeronave_9h-mlc',
       ...
       'destino_mx_amigos_y_familia', 'destino_playa',
       'origen_ciudad_fronteriza', 'origen_ciudad_principal',
       'origen_ecoturismo', 'origen_mx_amigos_y_familia', 'origen_playa',
       'Dia', 'DiasAFeriadoCercano', 'VacacionesEscolares'],
      dtype='object', length=189)